In [1]:
from pipline import Processing
import pandas as pd
import numpy as np
import joblib

In [2]:
df = pd.read_csv('data/raw/prueba_original.csv', delimiter=';')

In [3]:
p = Processing()

In [4]:
r = p.makebd(df)
lis = np.load('PeCat.npz')
pc = lis['arr_0']

In [5]:
r = p.standardized_texts(r, pc)

In [7]:
pp = p.bd_principal(r)
pp = p.standardized_columns(pp)
bd, bd_t = p.creation_bds(pp, r)

dga, oilp, oil_tap, oil_corr, insp, hump, hum_pap = bd
for i in range(0,len(bd)):
    base = bd[i]
    base.to_csv('data/processed/' + bd_t[i] + '.csv', sep = ';', index = False)

In [8]:
def test_preprocessing(df, secundarias, enc_perfil):
    
    copy = df
    if 'FECHA' in copy.columns:
        copy = copy.drop(columns=['FECHA'])

    for column in secundarias:
        ## checking_first
        try:
            if any(char.isdigit() for char in copy[copy[column].notna()][column].iloc[0]):

                copy[column] = copy[column].apply(column_transf)

            else:
                copy[column] = copy[column].isnull()
        
        except IndexError:
            copy[column] = False

    
    copy['perfil_catalogo'] = enc_perfil.transform(copy['perfil_catalogo'])

    return copy



def column_transf(x):
    '''Get first element of string. It must be calification o secundary column.'''
    if str(x)[0].isdigit():
        return str(x)[0]
    else: 
        return -1

In [9]:
enc_perfil = joblib.load('utils/encoder_perfil.pkl')
enc_avisos = joblib.load('utils/encoder_avisos.pkl')

rf_hump = joblib.load('models/rf_hump.pkl')
rf_humpap = joblib.load('models/rf_humpap.pkl')
rf_dga = joblib.load('models/rf_dga.pkl')
rf_insp = joblib.load('models/rf_insp.pkl')
rf_oiltap = joblib.load('models/rf_oiltap.pkl')
rf_oil = joblib.load('models/rf_oil.pkl')

hump_sec = ['DIAG', 'DELTA', 'SILI.GEL','T_M','FUG.ACEI','ING.ANIM','FUN.TERM','RES.CALEF','NIV.ACEI','SIST.ENF']
dga_sec = ['ARC','T.ACE','T.PAP','DP']
humpap_sec = ['NO.EQ','DELTA','DIAG','WHRT']
insp_sec = ['BUJ','ENF','M.OLTC','PRO-M','CUB','GAB','FUG.ACEI','COMP.ELEC','COMP.MEC','RES.CALEF']
oil_sec = ['DIE','FQ']
oiltap_sec = ['DIAG','DELTA-A','DELTA-B','DELTA-C']

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [11]:
test_humpap = pd.read_csv('data/processed/hum_pap.csv', delimiter=';')
thumpap_filt = test_preprocessing(test_humpap, secundarias = humpap_sec, enc_perfil=enc_perfil)
thumpap_filt = thumpap_filt.dropna(subset=['HUM_PAP'])
thumpap_filt = thumpap_filt[thumpap_filt['HUM_PAP'] <= 3]
ids = thumpap_filt['id_unique']
thumpap_filt = thumpap_filt.drop(columns=['id_unique'])
preds = rf_humpap.predict(thumpap_filt)
preds = enc_avisos.inverse_transform(preds)
humpap_preds = list(zip(preds, ids))
humpap_preds = pd.DataFrame(humpap_preds, columns=['pred','id'])

test_hump = pd.read_csv('data/processed/hump.csv', delimiter=';')
thump_filt = test_preprocessing(test_hump, secundarias = hump_sec, enc_perfil=enc_perfil)
thump_filt = thump_filt.dropna(subset=['HUMP'])
thump_filt = thump_filt[thump_filt['HUMP'] <= 3]
ids = thump_filt['id_unique']
thump_filt = thump_filt.drop(columns=['id_unique'])
preds = rf_hump.predict(thump_filt)
preds = enc_avisos.inverse_transform(preds)
hump_preds = list(zip(preds, ids))
hump_preds = pd.DataFrame(hump_preds, columns=['pred','id'])

test_dga = pd.read_csv('data/processed/dga.csv', delimiter=';')
tdga_filt = test_preprocessing(test_dga, secundarias = dga_sec, enc_perfil=enc_perfil)
tdga_filt = tdga_filt.dropna(subset=['DGA'])
tdga_filt = tdga_filt[tdga_filt['DGA'] <= 3]
ids = tdga_filt['id_unique']
tdga_filt = tdga_filt.drop(columns=['id_unique'])
preds = rf_dga.predict(tdga_filt)
preds = enc_avisos.inverse_transform(preds)
dga_preds = list(zip(preds, ids))
dga_preds = pd.DataFrame(dga_preds, columns=['pred','id'])

test_insp = pd.read_csv('data/processed/insp.csv', delimiter=';')
tinsp_filt = test_preprocessing(test_insp, secundarias = insp_sec, enc_perfil=enc_perfil)
tinsp_filt = tinsp_filt.dropna(subset=['INSP'])
tinsp_filt = tinsp_filt[tinsp_filt['INSP'] <= 3]
ids = tinsp_filt['id_unique']
tinsp_filt = tinsp_filt.drop(columns=['id_unique'])
preds = rf_insp.predict(tinsp_filt)
preds = enc_avisos.inverse_transform(preds)
insp_preds = list(zip(preds, ids))
insp_preds = pd.DataFrame(insp_preds, columns=['pred','id'])

test_oiltap = pd.read_csv('data/processed/oil_tap.csv', delimiter=';')
toiltap_filt = test_preprocessing(test_oiltap, secundarias = oiltap_sec, enc_perfil=enc_perfil)
toiltap_filt = toiltap_filt.dropna(subset=['OIL.TAP'])
toiltap_filt = toiltap_filt[toiltap_filt['OIL.TAP'] <= 3]
ids = toiltap_filt['id_unique']
toiltap_filt = toiltap_filt.drop(columns=['id_unique'])
preds = rf_oiltap.predict(toiltap_filt)
preds = enc_avisos.inverse_transform(preds)
oiltap_preds = list(zip(preds, ids))
oiltap_preds = pd.DataFrame(insp_preds, columns=['pred','id'])

test_oil = pd.read_csv('data/processed/oilp.csv', delimiter=';')
toil_filt = test_preprocessing(test_oil, secundarias = oil_sec, enc_perfil=enc_perfil)
toil_filt = toil_filt.dropna(subset=['OILP'])
toil_filt = toil_filt[toil_filt['OILP'] <= 3]
ids = toil_filt['id_unique']
toil_filt = toil_filt.drop(columns=['id_unique'])
preds = rf_oil.predict(toil_filt)
preds = enc_avisos.inverse_transform(preds)
oil_preds = list(zip(preds, ids))
oil_preds = pd.DataFrame(oil_preds, columns=['pred','id'])

In [12]:
pru = r
preds = humpap_preds.append([dga_preds, insp_preds, oiltap_preds,hump_preds,oil_preds])

ids = preds['id'].value_counts().index
res = []
for id in ids:
    p = list(preds[preds['id'] == id]['pred'])
    data = [id,p]
    res.append(data)
    
recomendaciones = pd.DataFrame(res,columns=['id_unique','recomendaciones'])

resultado = pru.set_index('id_unique').join(recomendaciones.set_index('id_unique'))

In [17]:
resultado.to_csv('data/recomendaciones/recs.csv')